In [2]:
import sqlite3
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re
import requests
import shutil

In [37]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("formula1.sqlite")
cur = con.cursor()

# Races

In [72]:
query = """
With R as (SELECT raceId, circuitId, year, date from races),
C as (SELECT circuitid, name, location, country, lat, lng from circuits)
Select * from R
INNER JOIN C ON R.circuitid = C.circuitid
order by year;
"""

In [73]:
df = pd.read_sql_query(query, con)
df

,raceId,circuitId,year,date,circuitid,name,location,country,lat,lng
0,833,9,1950,1950-05-13,9,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694
1,834,6,1950,1950-05-21,6,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056
2,835,19,1950,1950-05-30,19,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.23470
3,836,66,1950,1950-06-04,66,Circuit Bremgarten,Bern,Switzerland,46.9589,7.40194
4,837,13,1950,1950-06-18,13,Circuit de Spa-Francorchamps,Spa,Belgium,50.4372,5.97139
...,...,...,...,...,...,...,...,...,...,...
1035,1048,22,2020,2020-10-11,22,Suzuka Circuit,Suzuka,Japan,34.8431,136.54100
1036,1049,69,2020,2020-10-25,69,Circuit of the Americas,Austin,USA,30.1328,-97.64110
1037,1050,32,2020,2020-11-01,32,Autódromo Hermanos Rodríguez,Mexico City,Mexico,19.4042,-99.09070
1038,1051,18,2020,2020-11-15,18,Autódromo José Carlos Pace,São Paulo,Brazil,-23.7036,-46.69970


# Circuits

In [169]:
query = """
With R as (SELECT raceId, driverId, lap, position from lap_times),
C as (SELECT driverId, surname from drivers)
Select surname, lap, position from R
INNER JOIN C ON R.driverId == C.driverId
where raceid == 1030
order by position;
"""

In [170]:
df = pd.read_sql_query(query, con)
df

,surname,lap,position
0,Hamilton,1,1
1,Hamilton,2,1
2,Hamilton,3,1
3,Hamilton,4,1
4,Hamilton,5,1
...,...,...,...
1070,Gasly,41,20
1071,Gasly,42,20
1072,Gasly,43,20
1073,Gasly,44,20


In [175]:
drivers = df.groupby(by=['position'])
for driver in drivers:
    print(driver[0] , driver[1][driver[1]['lap'] == max(driver[1]['lap'])]['surname'].values[0], max(driver[1]['lap']))

1 Hamilton 55
2 Verstappen 55
3 Leclerc 55
4 Bottas 55
5 Vettel 55
6 Albon 55
7 Pérez 54
8 Norris 54
9 Kvyat 54
10 Sainz 54
11 Ricciardo 54
12 Hülkenberg 54
13 Räikkönen 54
14 Magnussen 54
15 Grosjean 54
16 Giovinazzi 54
17 Russell 54
18 Gasly 53
19 Kubica 53
20 Gasly 45


## Circuits plan

In [117]:
def get_html_page(url):
    html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [155]:
query = """
Select name, url from circuits
"""

In [156]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

logo = re.compile('(L|l)ogo')
path = 'https://en.wikipedia.org/'

for row in cur.execute(query):
    html = urlopen(row[1], context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")

    # Retrieve all of the anchor tags
    tmp = soup.findAll("table", {"class": "infobox vcard"})
    if len(tmp) > 0:
        length = tmp[0].findAll("a", {"class": "image"})
        if len(length) > 1:
            for img in length:
                filename = img['href'].lower()
                if filename.endswith('.svg') and not logo.search(filename):
                    src = img['href']
        elif len(length) == 1:
            src = length[0]['href']
    soup = get_html_page(path+src)
    img = 'https:'+soup.findAll("div", {'class': 'fullImageLink'})[0].findAll('a')[0]['href']
    print(img)
    if img.endswith('.png'):
        download_img(img, "img/circuits/{}.png".format(row[0]))              
    elif img.endswith('.svg'):
        download_img(img, "img/circuits/{}.svg".format(row[0]))      

https://upload.wikimedia.org/wikipedia/commons/3/31/Albert_Lake_Park_Street_Circuit_in_Melbourne%2C_Australia.svg
https://upload.wikimedia.org/wikipedia/commons/6/67/Sepang.svg
https://upload.wikimedia.org/wikipedia/commons/2/29/Bahrain_International_Circuit--Grand_Prix_Layout.svg
https://upload.wikimedia.org/wikipedia/commons/9/9c/Formula1_Circuit_Catalunya.svg
https://upload.wikimedia.org/wikipedia/commons/5/5d/Istanbul_park.svg
https://upload.wikimedia.org/wikipedia/commons/3/36/Monte_Carlo_Formula_1_track_map.svg
https://upload.wikimedia.org/wikipedia/commons/2/21/Circuit_Gilles_Villeneuve.svg
https://upload.wikimedia.org/wikipedia/commons/8/87/Circuit_de_Nevers_Magny-Cours.svg
https://upload.wikimedia.org/wikipedia/commons/1/1a/Silverstone_Circuit_2011.png
https://upload.wikimedia.org/wikipedia/commons/e/e5/Hockenheim2012.svg
https://upload.wikimedia.org/wikipedia/commons/9/91/Hungaroring.svg
https://upload.wikimedia.org/wikipedia/commons/0/0e/Circuit_Valensia_street.svg
https://u

# Constructors

In [10]:
path = 'https://www.formula1.com/'
url = 'https://www.formula1.com/en/teams.html'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

html = urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

In [11]:
tmp = soup.findAll("a", {"class": "listing-link"})

In [70]:
def download_img(url, filename):
    resp = requests.get(url, stream=True)
    # Open a local file with wb ( write binary ) permission.
    local_file = open(filename, 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)

In [13]:
for team in tmp:
    url = team['href']
    name = url.split('.')[0].split('/')[-1]
    html = urlopen(path+url, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    tmp2 = soup.findAll("img", {"class": "cq-dd-image"})
    download_img(path+tmp2[0]['src'],'logo_'+name+'.png')

# Drivers

In [144]:
query = """
Select driverId, nationality, country, country_code from drivers
"""

df = pd.read_sql_query(query, con)
df.head()

,driverId,nationality,country,country_code
0,1,British,United Kingdom,GBR
1,2,German,Germany,GER
2,3,German,Germany,GER
3,4,Spanish,Spain,ESP
4,5,Finnish,Finland,FIN


## Stats about Countries

In [145]:
countries = requests.get('https://raw.githubusercontent.com/mledoze/countries/master/countries.json').json()

In [146]:
nationalities_mapping = {}
for c in countries:
    if 'demonym' in c.keys() and c['independent'] == True:
        nationalities_mapping[c['demonym'].lower()] = {'country': c['name']["common"], 'off_code': c['cioc'], 'flag_code': c['cca3']}
    if 'demonyms' in c.keys() and c['independent'] == True:
        nationalities_mapping[c['demonyms']['eng']['m'].lower()] = {'country': c['name']["common"], 'off_code': c['cioc'], 'flag_code': c['cca3']}

In [140]:
# for key, value in sorted(nationalities_mapping.items(), key=lambda x: x[0]): 
#     print("{} : {}".format(key, value))

In [150]:
for row in df.values:
    tmp = nationalities_mapping.get(row[1].lower())
    if tmp is not None:
        name = tmp['country']
        code = tmp['flag_code']
    #     print("{} : {}".format(name, code))    
        url = "https://raw.githubusercontent.com/mledoze/countries/master/data/{}.svg".format(code.lower())
        obj = urlopen(url, context=ctx)
    #     print(obj.getcode())
        if obj.getcode() == 200:
            local_file = open("img/flags/{}.svg".format(name), 'wb')
            local_file.write(obj.read())
            local_file.close()

[340 'Rhodesian' 'None' 'None']
[368 'Rhodesian' 'None' 'None']
[414 'Rhodesian' 'None' 'None']
[473 'Rhodesian' 'None' 'None']
[491 'American-Italian' 'None' 'None']
[573 'Argentine-Italian' 'None' 'None']
[709 'East German' 'None' 'None']
[710 'East German' 'None' 'None']
[713 'East German' 'None' 'None']
